# Partie 3.3 - Détection d'anomalies

Ce notebook identifie les anomalies dans les données de consommation énergétique des bâtiments.  
Plusieurs méthodes complémentaires sont utilisées :
- Pics de consommation (3 écarts-types)
- Méthode IQR (interquartile range)
- Sous-consommation suspecte
- Incohérences DPE / consommation réelle
- Anomalies temporelles

**Source** : `../output/consommations_enrichies.parquet`

In [ ]:
# === Imports et chargement des données enrichies ===

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Chargement du fichier parquet enrichi
df = pd.read_parquet('../output/consommations_enrichies.parquet')

print(f"Nombre d'enregistrements : {len(df):,}")
print(f"Nombre de bâtiments     : {df['batiment_id'].nunique()}")
print(f"Période                 : {df['timestamp'].min()} → {df['timestamp'].max()}")
print(f"\nColonnes disponibles :")
print(df.columns.tolist())
df.head()

## 1. Pics de consommation anormaux (méthode des 3 écarts-types)

Pour chaque couple (bâtiment, type d'énergie), on calcule la moyenne et l'écart-type de la consommation.  
Tout enregistrement dont la consommation dépasse **moyenne + 3σ** est considéré comme un pic anormal.

In [ ]:
# === Détection des pics de consommation par la méthode 3-sigma ===

# Calcul de la moyenne et de l'écart-type par bâtiment et type d'énergie
stats_batiment = (
    df.groupby(['batiment_id', 'nom', 'type_energie'])['consommation']
    .agg(['mean', 'std'])
    .reset_index()
)
stats_batiment.columns = ['batiment_id', 'nom', 'type_energie', 'conso_mean', 'conso_std']

# Jointure pour ajouter les seuils au dataframe principal
df_sigma = df.merge(stats_batiment, on=['batiment_id', 'nom', 'type_energie'], how='left')

# Seuil haut : moyenne + 3 écarts-types
df_sigma['seuil_3sigma'] = df_sigma['conso_mean'] + 3 * df_sigma['conso_std']

# Flaguer les anomalies
df_sigma['anomalie_3sigma'] = df_sigma['consommation'] > df_sigma['seuil_3sigma']

anomalies_3sigma = df_sigma[df_sigma['anomalie_3sigma']].copy()

print(f"Nombre total d'anomalies détectées (3σ) : {len(anomalies_3sigma)}")
print(f"Pourcentage du jeu de données            : {len(anomalies_3sigma)/len(df)*100:.2f}%")
print()

# Top 20 des anomalies les plus importantes (écart le plus fort)
anomalies_3sigma['ecart_relatif'] = (
    (anomalies_3sigma['consommation'] - anomalies_3sigma['conso_mean'])
    / anomalies_3sigma['conso_std']
)

top20 = anomalies_3sigma.nlargest(20, 'ecart_relatif')[
    ['batiment_id', 'nom', 'type_energie', 'timestamp', 'consommation',
     'conso_mean', 'conso_std', 'ecart_relatif']
]

print("=== Top 20 des pics de consommation les plus anormaux ===")
display(top20)

# Nombre d'anomalies par bâtiment
anomalies_par_batiment_3sigma = (
    anomalies_3sigma.groupby(['batiment_id', 'nom'])
    .size()
    .reset_index(name='nb_anomalies_3sigma')
    .sort_values('nb_anomalies_3sigma', ascending=False)
)

print("\n=== Nombre d'anomalies 3σ par bâtiment ===")
display(anomalies_par_batiment_3sigma)

## 2. Méthode IQR pour la détection d'outliers

Méthode alternative basée sur l'écart interquartile (IQR = Q3 − Q1).  
Un outlier est défini comme toute valeur en dehors de l'intervalle **[Q1 − 1.5×IQR, Q3 + 1.5×IQR]**.

In [ ]:
# === Détection d'outliers par la méthode IQR ===

# Calcul des quartiles par bâtiment et type d'énergie
quartiles = (
    df.groupby(['batiment_id', 'nom', 'type_energie'])['consommation']
    .quantile([0.25, 0.75])
    .unstack()
    .reset_index()
)
quartiles.columns = ['batiment_id', 'nom', 'type_energie', 'Q1', 'Q3']
quartiles['IQR'] = quartiles['Q3'] - quartiles['Q1']
quartiles['seuil_bas_iqr'] = quartiles['Q1'] - 1.5 * quartiles['IQR']
quartiles['seuil_haut_iqr'] = quartiles['Q3'] + 1.5 * quartiles['IQR']

# Jointure avec le dataframe principal
df_iqr = df.merge(quartiles, on=['batiment_id', 'nom', 'type_energie'], how='left')

# Flaguer les outliers (hauts et bas)
df_iqr['outlier_iqr_haut'] = df_iqr['consommation'] > df_iqr['seuil_haut_iqr']
df_iqr['outlier_iqr_bas'] = df_iqr['consommation'] < df_iqr['seuil_bas_iqr']
df_iqr['outlier_iqr'] = df_iqr['outlier_iqr_haut'] | df_iqr['outlier_iqr_bas']

outliers_iqr = df_iqr[df_iqr['outlier_iqr']].copy()

print(f"Nombre d'outliers IQR détectés : {len(outliers_iqr)}")
print(f"  - Outliers hauts             : {df_iqr['outlier_iqr_haut'].sum()}")
print(f"  - Outliers bas               : {df_iqr['outlier_iqr_bas'].sum()}")
print(f"Pourcentage du jeu de données  : {len(outliers_iqr)/len(df)*100:.2f}%")
print()

# Comparaison des deux méthodes
print("=== Comparaison des méthodes de détection ===")
print(f"Anomalies 3-sigma : {len(anomalies_3sigma)}")
print(f"Outliers IQR      : {len(outliers_iqr)}")

# Identification des enregistrements détectés par les deux méthodes
idx_3sigma = set(anomalies_3sigma.index)
idx_iqr_haut = set(df_iqr[df_iqr['outlier_iqr_haut']].index)
communs = idx_3sigma & idx_iqr_haut
print(f"Détectés par les deux méthodes (pics hauts) : {len(communs)}")
print()

# Bâtiments avec le plus d'outliers IQR
outliers_par_batiment_iqr = (
    outliers_iqr.groupby(['batiment_id', 'nom'])
    .size()
    .reset_index(name='nb_outliers_iqr')
    .sort_values('nb_outliers_iqr', ascending=False)
)

print("=== Bâtiments avec le plus d'outliers IQR ===")
display(outliers_par_batiment_iqr.head(15))

## 3. Périodes de sous-consommation suspectes

Identification des périodes où un bâtiment présente une consommation anormalement basse  
(< moyenne − 2σ) **pendant les heures ouvrables** (jours de semaine, 8h-18h).  
Ces périodes peuvent indiquer des fermetures non signalées ou des pannes de compteur.

In [ ]:
# === Détection des périodes de sous-consommation suspectes ===

# Filtrer les heures ouvrables (lundi=0 à vendredi=4, 8h à 18h)
df_ouvrable = df.copy()
df_ouvrable['timestamp'] = pd.to_datetime(df_ouvrable['timestamp'])
df_ouvrable['jour_num'] = df_ouvrable['timestamp'].dt.dayofweek  # 0=lundi, 6=dimanche

mask_heures_ouvrables = (
    (df_ouvrable['jour_num'] < 5) &  # Lundi à vendredi
    (df_ouvrable['heure'] >= 8) &
    (df_ouvrable['heure'] <= 18)
)
df_ouvrable = df_ouvrable[mask_heures_ouvrables].copy()

# Calcul de la moyenne et de l'écart-type en heures ouvrables
stats_ouvrable = (
    df_ouvrable.groupby(['batiment_id', 'nom', 'type_energie'])['consommation']
    .agg(['mean', 'std'])
    .reset_index()
)
stats_ouvrable.columns = ['batiment_id', 'nom', 'type_energie', 'mean_ouvr', 'std_ouvr']

# Jointure et détection de sous-consommation
df_ouvrable = df_ouvrable.merge(stats_ouvrable, on=['batiment_id', 'nom', 'type_energie'], how='left')
df_ouvrable['seuil_bas_2sigma'] = df_ouvrable['mean_ouvr'] - 2 * df_ouvrable['std_ouvr']
df_ouvrable['sous_conso'] = df_ouvrable['consommation'] < df_ouvrable['seuil_bas_2sigma']

sous_conso = df_ouvrable[df_ouvrable['sous_conso']].copy()

print(f"Enregistrements en sous-consommation suspecte : {len(sous_conso)}")
print(f"Pourcentage des heures ouvrables              : {len(sous_conso)/len(df_ouvrable)*100:.2f}%")
print()

# Détection des périodes prolongées de quasi-zéro consommation (> 24h)
# On considère "quasi-zéro" comme < 5% de la moyenne
df_ouvrable['quasi_zero'] = df_ouvrable['consommation'] < (df_ouvrable['mean_ouvr'] * 0.05)

periodes_zero = []
for (bat_id, nom, energie), group in df_ouvrable.groupby(['batiment_id', 'nom', 'type_energie']):
    group = group.sort_values('timestamp')
    quasi_zero_mask = group['quasi_zero'].values
    timestamps = group['timestamp'].values
    
    if len(timestamps) == 0:
        continue
    
    # Identifier les séquences consécutives de quasi-zéro
    in_sequence = False
    debut = None
    
    for i in range(len(quasi_zero_mask)):
        if quasi_zero_mask[i] and not in_sequence:
            in_sequence = True
            debut = timestamps[i]
        elif not quasi_zero_mask[i] and in_sequence:
            in_sequence = False
            fin = timestamps[i - 1]
            duree_heures = (pd.Timestamp(fin) - pd.Timestamp(debut)).total_seconds() / 3600
            if duree_heures > 24:
                periodes_zero.append({
                    'batiment_id': bat_id,
                    'nom': nom,
                    'type_energie': energie,
                    'debut': debut,
                    'fin': fin,
                    'duree_heures': round(duree_heures, 1)
                })
    
    # Vérifier la dernière séquence
    if in_sequence:
        fin = timestamps[-1]
        duree_heures = (pd.Timestamp(fin) - pd.Timestamp(debut)).total_seconds() / 3600
        if duree_heures > 24:
            periodes_zero.append({
                'batiment_id': bat_id,
                'nom': nom,
                'type_energie': energie,
                'debut': debut,
                'fin': fin,
                'duree_heures': round(duree_heures, 1)
            })

df_periodes_zero = pd.DataFrame(periodes_zero)

if len(df_periodes_zero) > 0:
    df_periodes_zero = df_periodes_zero.sort_values('duree_heures', ascending=False)
    print(f"Périodes prolongées (>24h) de quasi-zéro consommation : {len(df_periodes_zero)}")
    display(df_periodes_zero.head(20))
else:
    print("Aucune période prolongée de quasi-zéro consommation détectée.")

# Bâtiments les plus touchés par la sous-consommation
sous_conso_par_bat = (
    sous_conso.groupby(['batiment_id', 'nom'])
    .size()
    .reset_index(name='nb_sous_conso')
    .sort_values('nb_sous_conso', ascending=False)
)
print("\n=== Bâtiments les plus touchés par la sous-consommation ===")
display(sous_conso_par_bat.head(10))

## 4. Incohérences entre DPE et consommation réelle

Comparaison de la consommation réelle par m² avec les plages attendues  
selon la classe énergétique (DPE) du bâtiment.

| Classe | Plage (kWh/m²/an) |
|--------|-------------------|
| A      | < 70              |
| B      | 70 – 110          |
| C      | 110 – 180         |
| D      | 180 – 250         |
| E      | 250 – 330         |
| F      | 330 – 420         |
| G      | > 420             |

In [ ]:
# === Incohérences entre classe DPE et consommation réelle ===

# Plages de référence DPE (kWh/m²/an)
dpe_ranges = {
    'A': (0, 70),
    'B': (70, 110),
    'C': (110, 180),
    'D': (180, 250),
    'E': (250, 330),
    'F': (330, 420),
    'G': (420, float('inf'))
}

# Calcul de la consommation annuelle par m² pour chaque bâtiment
# On agrège par bâtiment pour obtenir la consommation totale annuelle
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['annee'] = df['timestamp'].dt.year

conso_annuelle = (
    df.groupby(['batiment_id', 'nom', 'classe_energetique', 'surface_m2', 'annee'])
    .agg(conso_totale=('consommation', 'sum'))
    .reset_index()
)

# Consommation par m² annuelle
conso_annuelle['conso_par_m2_an'] = conso_annuelle['conso_totale'] / conso_annuelle['surface_m2']

# Moyenne sur les années disponibles
conso_moyenne_bat = (
    conso_annuelle.groupby(['batiment_id', 'nom', 'classe_energetique', 'surface_m2'])
    .agg(conso_par_m2_an_moy=('conso_par_m2_an', 'mean'))
    .reset_index()
)

# Vérification de la cohérence avec la classe DPE
def verifier_coherence_dpe(row):
    """Vérifie si la consommation réelle est cohérente avec la classe DPE."""
    classe = row['classe_energetique']
    conso = row['conso_par_m2_an_moy']
    
    if classe not in dpe_ranges:
        return 'classe_inconnue'
    
    borne_basse, borne_haute = dpe_ranges[classe]
    
    if conso < borne_basse:
        return 'sous_classe'  # Consomme moins que sa classe
    elif conso > borne_haute:
        return 'sur_classe'   # Consomme plus que sa classe
    else:
        return 'coherent'

def trouver_classe_reelle(conso):
    """Détermine la classe DPE réelle en fonction de la consommation."""
    for classe, (borne_basse, borne_haute) in dpe_ranges.items():
        if borne_basse <= conso < borne_haute:
            return classe
    return 'G'

conso_moyenne_bat['coherence_dpe'] = conso_moyenne_bat.apply(verifier_coherence_dpe, axis=1)
conso_moyenne_bat['classe_reelle'] = conso_moyenne_bat['conso_par_m2_an_moy'].apply(trouver_classe_reelle)

# Résultats
print("=== Cohérence DPE / Consommation réelle ===")
print(conso_moyenne_bat['coherence_dpe'].value_counts())
print()

# Bâtiments incohérents
incoherents = conso_moyenne_bat[conso_moyenne_bat['coherence_dpe'] != 'coherent'].copy()
incoherents['ecart_classe'] = incoherents.apply(
    lambda r: ord(r['classe_reelle']) - ord(r['classe_energetique']),
    axis=1
)

print(f"Nombre de bâtiments avec incohérence DPE : {len(incoherents)}")
print()

if len(incoherents) > 0:
    incoherents_sorted = incoherents.sort_values('ecart_classe', ascending=False)
    print("=== Bâtiments dont la consommation ne correspond pas au DPE ===")
    display(
        incoherents_sorted[[
            'batiment_id', 'nom', 'classe_energetique', 'classe_reelle',
            'conso_par_m2_an_moy', 'surface_m2', 'coherence_dpe', 'ecart_classe'
        ]]
    )
else:
    print("Tous les bâtiments sont cohérents avec leur classe DPE.")

## 5. Anomalies temporelles

Détection de schémas de consommation inhabituels :
- Consommation pendant les heures de fermeture connues
- Consommation le week-end pour les bâtiments censés être fermés (écoles, mairies)
- Bâtiments avec des profils incohérents

In [ ]:
# === Détection d'anomalies temporelles ===

df_temp = df.copy()
df_temp['timestamp'] = pd.to_datetime(df_temp['timestamp'])
df_temp['jour_num'] = df_temp['timestamp'].dt.dayofweek  # 0=lundi, 6=dimanche
df_temp['est_weekend'] = df_temp['jour_num'] >= 5

# Types de bâtiments normalement fermés le week-end
types_fermes_weekend = ['ecole', 'école', 'mairie', 'college', 'collège',
                        'lycee', 'lycée', 'creche', 'crèche', 'bibliotheque',
                        'bibliothèque', 'administration']

# Identifier les bâtiments censés être fermés le week-end (par type)
df_temp['type_lower'] = df_temp['type'].str.lower().str.strip()
df_temp['devrait_fermer_weekend'] = df_temp['type_lower'].isin(types_fermes_weekend)

# Calculer la consommation moyenne en semaine vs week-end par bâtiment
conso_semaine = (
    df_temp[~df_temp['est_weekend']]
    .groupby(['batiment_id', 'nom', 'type'])['consommation']
    .mean()
    .reset_index(name='conso_moy_semaine')
)

conso_weekend = (
    df_temp[df_temp['est_weekend']]
    .groupby(['batiment_id', 'nom', 'type'])['consommation']
    .mean()
    .reset_index(name='conso_moy_weekend')
)

comparaison_temporelle = conso_semaine.merge(conso_weekend, on=['batiment_id', 'nom', 'type'], how='inner')
comparaison_temporelle['ratio_weekend_semaine'] = (
    comparaison_temporelle['conso_moy_weekend'] / comparaison_temporelle['conso_moy_semaine']
)

# Bâtiments devant être fermés le week-end mais avec forte consommation
comparaison_temporelle['type_lower'] = comparaison_temporelle['type'].str.lower().str.strip()
comparaison_temporelle['devrait_fermer_weekend'] = (
    comparaison_temporelle['type_lower'].isin(types_fermes_weekend)
)

# Seuil : si la consommation week-end dépasse 50% de celle de la semaine
# pour un bâtiment qui devrait être fermé → anomalie
anomalies_weekend = comparaison_temporelle[
    (comparaison_temporelle['devrait_fermer_weekend']) &
    (comparaison_temporelle['ratio_weekend_semaine'] > 0.5)
].copy()

print(f"Bâtiments avec consommation week-end suspecte : {len(anomalies_weekend)}")
if len(anomalies_weekend) > 0:
    display(
        anomalies_weekend[[
            'batiment_id', 'nom', 'type', 'conso_moy_semaine',
            'conso_moy_weekend', 'ratio_weekend_semaine'
        ]].sort_values('ratio_weekend_semaine', ascending=False)
    )
print()

# Détection de consommation nocturne anormale (22h-6h) pour tous les bâtiments
df_temp['est_nuit'] = (df_temp['heure'] >= 22) | (df_temp['heure'] < 6)

conso_jour = (
    df_temp[~df_temp['est_nuit']]
    .groupby(['batiment_id', 'nom', 'type'])['consommation']
    .mean()
    .reset_index(name='conso_moy_jour')
)

conso_nuit = (
    df_temp[df_temp['est_nuit']]
    .groupby(['batiment_id', 'nom', 'type'])['consommation']
    .mean()
    .reset_index(name='conso_moy_nuit')
)

comparaison_nuit = conso_jour.merge(conso_nuit, on=['batiment_id', 'nom', 'type'], how='inner')
comparaison_nuit['ratio_nuit_jour'] = (
    comparaison_nuit['conso_moy_nuit'] / comparaison_nuit['conso_moy_jour']
)

# Bâtiments avec consommation nocturne excessive (>70% de la journée)
anomalies_nuit = comparaison_nuit[comparaison_nuit['ratio_nuit_jour'] > 0.7].copy()

print(f"Bâtiments avec consommation nocturne anormalement élevée : {len(anomalies_nuit)}")
if len(anomalies_nuit) > 0:
    display(
        anomalies_nuit[[
            'batiment_id', 'nom', 'type', 'conso_moy_jour',
            'conso_moy_nuit', 'ratio_nuit_jour'
        ]].sort_values('ratio_nuit_jour', ascending=False)
    )

## 6. Score d'anomalie global par bâtiment

Construction d'un score composite basé sur :
- Nombre de pics de consommation (3σ)
- Nombre d'outliers IQR
- Incohérence DPE
- Anomalies temporelles (week-end, nuit)

In [ ]:
# === Construction du score d'anomalie global par bâtiment ===

# Base : liste de tous les bâtiments
batiments = df[['batiment_id', 'nom', 'type', 'commune', 'surface_m2', 'classe_energetique']].drop_duplicates()

# 1. Score pics 3-sigma (normalisé)
score_3sigma = anomalies_par_batiment_3sigma.copy()
if len(score_3sigma) > 0 and score_3sigma['nb_anomalies_3sigma'].max() > 0:
    score_3sigma['score_3sigma'] = (
        score_3sigma['nb_anomalies_3sigma'] / score_3sigma['nb_anomalies_3sigma'].max() * 25
    )
else:
    score_3sigma['score_3sigma'] = 0

# 2. Score outliers IQR (normalisé)
score_iqr = outliers_par_batiment_iqr.copy()
if len(score_iqr) > 0 and score_iqr['nb_outliers_iqr'].max() > 0:
    score_iqr['score_iqr'] = (
        score_iqr['nb_outliers_iqr'] / score_iqr['nb_outliers_iqr'].max() * 25
    )
else:
    score_iqr['score_iqr'] = 0

# 3. Score incohérence DPE
score_dpe = conso_moyenne_bat[['batiment_id', 'nom', 'coherence_dpe']].copy()
score_dpe['score_dpe'] = score_dpe['coherence_dpe'].map({
    'coherent': 0,
    'sous_classe': 10,
    'sur_classe': 25,
    'classe_inconnue': 5
})

# 4. Score anomalies temporelles
score_weekend = anomalies_weekend[['batiment_id', 'nom', 'ratio_weekend_semaine']].copy() if len(anomalies_weekend) > 0 else pd.DataFrame(columns=['batiment_id', 'nom', 'ratio_weekend_semaine'])
score_weekend['score_weekend'] = np.minimum(score_weekend['ratio_weekend_semaine'] * 25, 25) if len(score_weekend) > 0 else 0

score_nuit_df = anomalies_nuit[['batiment_id', 'nom', 'ratio_nuit_jour']].copy() if len(anomalies_nuit) > 0 else pd.DataFrame(columns=['batiment_id', 'nom', 'ratio_nuit_jour'])
score_nuit_df['score_nuit'] = np.minimum(score_nuit_df['ratio_nuit_jour'] * 15, 15) if len(score_nuit_df) > 0 else 0

# Assemblage du score global
score_global = batiments.copy()

# Joindre les différents scores
score_global = score_global.merge(
    score_3sigma[['batiment_id', 'score_3sigma']],
    on='batiment_id', how='left'
)
score_global = score_global.merge(
    score_iqr[['batiment_id', 'score_iqr']],
    on='batiment_id', how='left'
)
score_global = score_global.merge(
    score_dpe[['batiment_id', 'score_dpe']],
    on='batiment_id', how='left'
)
score_global = score_global.merge(
    score_weekend[['batiment_id', 'score_weekend']],
    on='batiment_id', how='left'
)
score_global = score_global.merge(
    score_nuit_df[['batiment_id', 'score_nuit']],
    on='batiment_id', how='left'
)

# Remplir les valeurs manquantes par 0
cols_score = ['score_3sigma', 'score_iqr', 'score_dpe', 'score_weekend', 'score_nuit']
score_global[cols_score] = score_global[cols_score].fillna(0)

# Score total (max théorique = 100)
score_global['score_anomalie'] = score_global[cols_score].sum(axis=1).round(1)

# Classification par niveau de risque
def classifier_risque(score):
    """Classe le niveau de risque en fonction du score d'anomalie."""
    if score >= 60:
        return 'CRITIQUE'
    elif score >= 40:
        return 'ELEVE'
    elif score >= 20:
        return 'MOYEN'
    elif score >= 5:
        return 'FAIBLE'
    else:
        return 'NORMAL'

score_global['niveau_risque'] = score_global['score_anomalie'].apply(classifier_risque)

# Classement
score_global = score_global.sort_values('score_anomalie', ascending=False).reset_index(drop=True)

print("=== Score d'anomalie global par bâtiment ===")
print(f"\nRépartition par niveau de risque :")
print(score_global['niveau_risque'].value_counts())
print()

display(
    score_global[[
        'batiment_id', 'nom', 'type', 'commune', 'surface_m2',
        'classe_energetique', 'score_3sigma', 'score_iqr', 'score_dpe',
        'score_weekend', 'score_nuit', 'score_anomalie', 'niveau_risque'
    ]].head(20)
)

## 7. Liste des bâtiments nécessitant un audit énergétique

Priorisation des bâtiments pour un audit en fonction du score d'anomalie,  
de l'incohérence DPE et du niveau de consommation.

In [ ]:
# === Liste priorisée des bâtiments nécessitant un audit énergétique ===

# Enrichir avec la consommation moyenne par m²
audit_list = score_global.merge(
    conso_moyenne_bat[['batiment_id', 'conso_par_m2_an_moy', 'classe_reelle']],
    on='batiment_id', how='left'
)

# Générer des recommandations personnalisées pour chaque bâtiment
def generer_recommandation(row):
    """Génère une recommandation d'audit personnalisée."""
    recommandations = []
    
    if row['score_dpe'] >= 20:
        recommandations.append(
            f"Reclassification DPE urgente : classe déclarée {row['classe_energetique']}, "
            f"classe réelle estimée {row.get('classe_reelle', 'N/A')}"
        )
    
    if row['score_3sigma'] >= 15:
        recommandations.append(
            "Vérification des équipements : pics de consommation fréquents détectés"
        )
    
    if row['score_weekend'] >= 10:
        recommandations.append(
            "Audit de la programmation horaire : consommation week-end anormale"
        )
    
    if row['score_nuit'] >= 8:
        recommandations.append(
            "Contrôle de l'éclairage et du chauffage nocturne : consommation de nuit excessive"
        )
    
    if row['score_iqr'] >= 15:
        recommandations.append(
            "Analyse approfondie du profil de consommation : nombreuses valeurs aberrantes"
        )
    
    if not recommandations:
        recommandations.append("Audit de routine recommandé")
    
    return ' | '.join(recommandations)

audit_list['recommandation'] = audit_list.apply(generer_recommandation, axis=1)

# Définir la priorité d'audit
def definir_priorite(row):
    """Définit la priorité d'audit selon le score et les caractéristiques."""
    if row['niveau_risque'] == 'CRITIQUE':
        return 1  # Priorité maximale
    elif row['niveau_risque'] == 'ELEVE':
        return 2
    elif row['niveau_risque'] == 'MOYEN' and row['score_dpe'] > 0:
        return 2  # Rehaussé si incohérence DPE
    elif row['niveau_risque'] == 'MOYEN':
        return 3
    else:
        return 4

audit_list['priorite_audit'] = audit_list.apply(definir_priorite, axis=1)

# Filtrer les bâtiments nécessitant un audit (score > 5)
audit_necessaire = audit_list[audit_list['score_anomalie'] >= 5].sort_values(
    ['priorite_audit', 'score_anomalie'],
    ascending=[True, False]
).reset_index(drop=True)

print(f"=== Bâtiments nécessitant un audit énergétique : {len(audit_necessaire)} ===")
print(f"\nRépartition par priorité :")
print(audit_necessaire['priorite_audit'].value_counts().sort_index())
print()

# Affichage détaillé
colonnes_affichage = [
    'priorite_audit', 'batiment_id', 'nom', 'type', 'commune',
    'classe_energetique', 'conso_par_m2_an_moy', 'score_anomalie',
    'niveau_risque', 'recommandation'
]
colonnes_disponibles = [c for c in colonnes_affichage if c in audit_necessaire.columns]
display(audit_necessaire[colonnes_disponibles])

print(f"\n--- Priorité 1 (Critique) : {len(audit_necessaire[audit_necessaire['priorite_audit'] == 1])} bâtiments")
print(f"--- Priorité 2 (Élevée)   : {len(audit_necessaire[audit_necessaire['priorite_audit'] == 2])} bâtiments")
print(f"--- Priorité 3 (Moyenne)   : {len(audit_necessaire[audit_necessaire['priorite_audit'] == 3])} bâtiments")
print(f"--- Priorité 4 (Faible)    : {len(audit_necessaire[audit_necessaire['priorite_audit'] == 4])} bâtiments")

## 8. Export des anomalies

Sauvegarde de toutes les anomalies détectées dans un fichier CSV structuré.

In [ ]:
# === Export des anomalies détectées ===

anomalies_export = []

# 1. Anomalies 3-sigma (pics de consommation)
for _, row in anomalies_3sigma.iterrows():
    anomalies_export.append({
        'batiment_id': row['batiment_id'],
        'nom': row['nom'],
        'type_anomalie': 'pic_consommation_3sigma',
        'description': (
            f"Consommation de {row['consommation']:.1f} kWh détectée le {row['timestamp']} "
            f"({row['type_energie']}), soit {row['ecart_relatif']:.1f} écarts-types au-dessus de la moyenne"
        ),
        'severite': 'haute' if row['ecart_relatif'] > 5 else 'moyenne',
        'recommandation': 'Vérifier les équipements et la régulation du bâtiment'
    })

# 2. Incohérences DPE
if len(incoherents) > 0:
    for _, row in incoherents.iterrows():
        severite = 'haute' if abs(row['ecart_classe']) >= 2 else 'moyenne'
        anomalies_export.append({
            'batiment_id': row['batiment_id'],
            'nom': row['nom'],
            'type_anomalie': 'incoherence_dpe',
            'description': (
                f"Classe DPE déclarée : {row['classe_energetique']}, "
                f"classe réelle estimée : {row['classe_reelle']} "
                f"(consommation réelle : {row['conso_par_m2_an_moy']:.0f} kWh/m²/an)"
            ),
            'severite': severite,
            'recommandation': 'Reclassification DPE et audit énergétique complet'
        })

# 3. Anomalies week-end
if len(anomalies_weekend) > 0:
    for _, row in anomalies_weekend.iterrows():
        anomalies_export.append({
            'batiment_id': row['batiment_id'],
            'nom': row['nom'],
            'type_anomalie': 'consommation_weekend_suspecte',
            'description': (
                f"Consommation week-end = {row['ratio_weekend_semaine']*100:.0f}% "
                f"de la consommation en semaine pour un bâtiment de type '{row['type']}'"
            ),
            'severite': 'haute' if row['ratio_weekend_semaine'] > 0.8 else 'moyenne',
            'recommandation': 'Vérifier la programmation horaire et les équipements en veille'
        })

# 4. Anomalies nocturnes
if len(anomalies_nuit) > 0:
    for _, row in anomalies_nuit.iterrows():
        anomalies_export.append({
            'batiment_id': row['batiment_id'],
            'nom': row['nom'],
            'type_anomalie': 'consommation_nocturne_excessive',
            'description': (
                f"Consommation nocturne = {row['ratio_nuit_jour']*100:.0f}% "
                f"de la consommation diurne"
            ),
            'severite': 'haute' if row['ratio_nuit_jour'] > 0.9 else 'moyenne',
            'recommandation': 'Contrôler éclairage, chauffage et équipements actifs la nuit'
        })

# 5. Sous-consommation suspecte (périodes prolongées)
if len(df_periodes_zero) > 0:
    for _, row in df_periodes_zero.iterrows():
        anomalies_export.append({
            'batiment_id': row['batiment_id'],
            'nom': row['nom'],
            'type_anomalie': 'sous_consommation_prolongee',
            'description': (
                f"Quasi-zéro consommation ({row['type_energie']}) pendant {row['duree_heures']}h "
                f"du {row['debut']} au {row['fin']}"
            ),
            'severite': 'haute' if row['duree_heures'] > 72 else 'moyenne',
            'recommandation': 'Vérifier le compteur et les raisons de la fermeture éventuelle'
        })

# Création du DataFrame d'export
df_anomalies = pd.DataFrame(anomalies_export)

if len(df_anomalies) > 0:
    # Ordonner par sévérité puis par bâtiment
    ordre_severite = {'haute': 0, 'moyenne': 1, 'basse': 2}
    df_anomalies['ordre_sev'] = df_anomalies['severite'].map(ordre_severite)
    df_anomalies = df_anomalies.sort_values(['ordre_sev', 'batiment_id']).drop(columns='ordre_sev')
    df_anomalies = df_anomalies.reset_index(drop=True)

# Sauvegarde en CSV
chemin_export = '../output/anomalies_detectees.csv'
df_anomalies.to_csv(chemin_export, index=False, encoding='utf-8-sig')

print(f"Fichier exporté : {chemin_export}")
print(f"Nombre total d'anomalies exportées : {len(df_anomalies)}")
print()

# Statistiques récapitulatives
print("=== Résumé des anomalies détectées ===")
print(f"\nPar type d'anomalie :")
if len(df_anomalies) > 0:
    print(df_anomalies['type_anomalie'].value_counts().to_string())
    print(f"\nPar sévérité :")
    print(df_anomalies['severite'].value_counts().to_string())
    print(f"\nNombre de bâtiments concernés : {df_anomalies['batiment_id'].nunique()}")
else:
    print("Aucune anomalie détectée.")

print("\n=== Aperçu du fichier exporté ===")
display(df_anomalies.head(20))

## Rapport de recommandations d'audit

### Synthèse des résultats

L'analyse de détection d'anomalies a identifié plusieurs catégories de problèmes dans les données de consommation énergétique des bâtiments publics :

---

### 1. Bâtiments prioritaires pour un audit (Priorité 1 - CRITIQUE)

Ces bâtiments présentent des anomalies multiples et sévères nécessitant une intervention **immédiate** :
- Pics de consommation récurrents dépassant 3 écarts-types
- Forte incohérence entre la classe DPE déclarée et la consommation réelle
- Consommation anormale pendant les heures de fermeture

**Action recommandée** : Audit énergétique complet dans les 3 mois

---

### 2. Bâtiments sous surveillance (Priorité 2 - ÉLEVÉE)

Ces bâtiments montrent des signes significatifs d'inefficacité ou d'anomalies :
- Consommation week-end supérieure à 50% de la consommation en semaine
- Écart notable entre DPE déclaré et consommation mesurée
- Profils de consommation nocturne inhabituels

**Action recommandée** : Audit ciblé dans les 6 mois, vérification de la programmation horaire

---

### 3. Bâtiments à surveiller (Priorité 3 - MOYENNE)

Ces bâtiments présentent des anomalies ponctuelles ou modérées :
- Quelques pics de consommation isolés
- Légère incohérence DPE

**Action recommandée** : Surveillance renforcée, audit de routine dans l'année

---

### 4. Recommandations transversales

| Domaine | Recommandation |
|---------|---------------|
| **Compteurs** | Vérifier l'étalonnage des compteurs pour les bâtiments avec sous-consommation prolongée |
| **Programmation** | Auditer les systèmes de gestion technique (GTC/GTB) pour les bâtiments avec consommation hors horaires |
| **DPE** | Lancer une campagne de reclassification DPE pour les bâtiments identifiés comme incohérents |
| **Équipements** | Inspecter les équipements (chauffage, climatisation) des bâtiments avec pics récurrents |
| **Suivi** | Mettre en place un tableau de bord de suivi continu avec alertes automatiques |

---

### Fichiers générés

- `../output/anomalies_detectees.csv` : Liste détaillée de toutes les anomalies avec recommandations

---

*Rapport généré automatiquement — ECF Data Engineer — Analyse énergétique des bâtiments publics*